In [14]:
import pandas as pd
import re
import numpy as np

df = pd.read_csv("paper_data.csv")
df.head()

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


In [15]:
def compute_average(group_name, df):
    group_columns = [col for col in df.columns if group_name in col]
    return df[group_columns].mean(axis=1)

# Initialize a new DataFrame to store the average values
paper_df = pd.DataFrame()

# Iterate through each group and compute the average
groups = ['alpha', 'beta', 'delta', 'theta']
for group in groups:
    paper_df[f'{group}'] = compute_average(group, df)

paper_df.head()

,alpha,beta,delta,theta
0,35.761823,17.688020,16.911349,27.942618
1,42.364701,23.502045,31.233618,34.165601
2,30.072722,27.437001,27.230425,28.751071
3,49.038828,29.970119,17.596805,29.102476
4,45.296634,44.180980,25.211048,34.791592


In [16]:
david_test_brain  = pd.read_csv("david_sample.csv")
david_test_brain.head()

,Timestamp (ms),ch0_1Hz,ch0_2Hz,ch0_3Hz,ch0_4Hz,ch0_5Hz,ch0_6Hz,ch0_7Hz,ch0_8Hz,ch0_9Hz,...,f_22Hz,f_23Hz,f_24Hz,f_25Hz,f_26Hz,f_27Hz,f_28Hz,f_29Hz,f_30Hz,info
0,1697308927358,8.237185,104.395855,51.182814,49.070749,65.770352,41.711812,7.156551,12.943227,13.118700,...,22,23,24,25,26,27,28,29,30,[object Object]
1,1697308927777,23.636800,83.028761,50.256473,53.076349,10.250856,21.606699,6.856555,11.531993,3.747221,...,22,23,24,25,26,27,28,29,30,[object Object]
2,1697308928153,12.863504,35.215964,97.360086,23.417080,19.258428,20.020059,7.976734,12.120806,5.940289,...,22,23,24,25,26,27,28,29,30,[object Object]
3,1697308928527,10.187540,47.298000,105.042990,26.348256,22.559335,8.440813,8.752169,0.372770,3.035038,...,22,23,24,25,26,27,28,29,30,[object Object]
4,1697308928948,29.110834,95.514180,73.526746,55.985294,20.699925,31.093223,25.550001,21.309790,10.638872,...,22,23,24,25,26,27,28,29,30,[object Object]


In [17]:
def consolidate(freq_min, freq_max, brainwave, userdf, df_temp):
    
    pattern = re.compile(rf".*[{freq_min}-{freq_max}]Hz.*")
    columns_to_include = [col for col in userdf.columns if re.match(pattern, col)]

    bw_average = userdf[columns_to_include].mean(axis=1)
    bw_median = np.array([np.median(bw_average)])

    df_temp[f"{brainwave}"] = bw_median

In [19]:
df_temp = pd.DataFrame()
df = pd.DataFrame()

pattern = re.compile("^Aux|^f_")
unwanted = [col for col in david_test_brain.columns if re.match(pattern, col)]
userdf = david_test_brain.drop(columns=unwanted)
userdf = userdf.drop('info', axis=1)

consolidate(1, 3, "Delta", userdf, df_temp)
consolidate(4, 7, "Theta", userdf, df_temp)
consolidate(8, 9, "Alpha1", userdf, df_temp)
consolidate(10, 11, "Alpha2", userdf, df_temp)
consolidate(12, 20, "Beta1", userdf, df_temp)
consolidate(20, 29, "Beta2", userdf, df_temp)

# combine alpha1 + alpha2, and beta1 + beta2
groups = ['Alpha', 'Beta']
for group in groups:
    df_temp[f'{group}'] = compute_average(group, df_temp)

userdf = df_temp
userdf = userdf.drop(columns=['Alpha1', 'Alpha2', 'Beta1', 'Beta2'])

userdf.head()

,Delta,Theta,Alpha,Beta
0,26.542049,15.867994,12.056772,18.641851
